<a href="https://colab.research.google.com/github/mikewwalker-gh/topic-finder-aid/blob/main/nGramProc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import os
import random
from datetime import datetime
import string
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.util import ngrams
import matplotlib.pyplot as plt
import csv
import re


In [ ]:
# Read  all or read a sample of  random text files from directory path into a df
directory_path = '/opt/app/agent-assist/data/chat/transcripts/2022-10'
df = read_text_files_to_df(directory_path)  # Read all files into a DataFrame
# df = read_text_files_to_df(directory_path, num_files_to_sample=100)  # Read a random sample of 10,000 files into a DataFrame

In [ ]:
# preprocess df['topic']

In [ ]:
# Create another column in df, df'['clean_topic'] and call function clean_column() to clean repetitive, noisy strings.
# df['clean_chat'] = clean_column(df, 'chat')
df['clean_topic'] = clean_column(df, 'topic')

In [ ]:
# another step in cleaning, remove strings matching regex patterns
df['clean_topic'] = apply_regex_replacements(df, 'clean_topic', regex_patterns)

In [ ]:
# remove punctuation from df['clean_topic']
df['clean_topic'] = df['clean_topic'].str.replace('[^\w\s]', '')

In [ ]:
# remove occurance of one or more spaces with a single space in df['clean_topic']
df['clean_topic'] = df['clean_topic'].str.replace(' +', ' ')

In [ ]:
# remove punctuation and digits, split the text into words, filter out non-alpha words and rejoin the words.
df['clean_topic'] = df['clean_topic'].apply(clean_text)

In [ ]:
# compare size of all content in all rows of column 'topic' in df to size of all text in column 'clean_topic' in df

print(df['topic'].str.len().sum())
print(df['clean_topic'].str.len().sum())

In [ ]:
# Define stop words
stop_words = set(stopwords.words('english'))

In [ ]:
# tokenize and fileter for stop words, also convert to lower case before processing.

df['topic_tokens'] = df['clean_topic'].apply(tokenize_and_filter)

In [ ]:
# Add ngrams in columns - unigrams, bigrams, trigrams and quadgrams to the df.
df = add_ngram_columns(df)

In [ ]:
# Read df here, and them lemmatize 'clean_topic' and all ngram columns; Also
# remove the max_rows_per_ngram restriction when calling create_training_data
# function below. %% Assuming df has 'unigrams', 'bigrams', 'trigrams',
# 'quadgrams' columns

In [ ]:
# Assuming df has 'unigrams', 'bigrams', 'trigrams', 'quadgrams' columns
freq_dists = compute_ngram_freq_dist(df, 'unigrams', 'bigrams', 'trigrams', 'quadgrams')

In [ ]:
# Create training data using df, freq_dists and keywords from taxonomy:
# df = your dataframe with columns: 'filename', 'topic', 'unigrams', 'bigrams', 'trigrams', 'quadgrams'
# freq_dists = your frequency distributions for 'unigrams', 'bigrams', 'trigrams', 'quadgrams'
keywords = ['claim', 'coverage','enrollment','medicare','medicaregov','premium','social security']
training_df = create_training_data(df, freq_dists, keywords, top_n=500, max_rows_per_ngram=25)
# print(training_df)

In [ ]:
# for word, frequency in freq_dists['unigrams'].most_common(20):
#     print(word, frequency)
# len(freq_dists['quadgrams'])
freq_dists['trigrams'].most_common(5)


In [ ]:
# pd.set_option('display.max_rows', None)  # Display all rows
# training_df.sort_values(by='Keyword',ascending=True)
training_df.to_csv('training_df_500_25b')

In [ ]:
############################################## Run Functions and set variables; do this first ##############################
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#

In [ ]:
regex_patterns = [
    r'My name is \[.*?\]',  # Pattern to match 'My name is [Agent Name]'
    r'Party \[.*?\] left the session',  # Pattern to match 'Party [Agent Name] left the session'
    r'\[.*?\] has joined the chat',  # Pattern to match '[Agent Name] has joined the chat'
    r'you have a nice (day|night|afternoon)',  # Pattern to match 'day, night, afternoon'
    r'Please do not give me (his|her) Medicare number',
    r'\[.*?\] has left the chat',  # Pattern to match '[Agent Name] has left the chat'
    r'my name is ([a-zA-Z]+(?: [a-zA-Z]+)*)\.',  # Pattern to match 'My name is [Name].'
    r'\[.*?\] is typing',  # Pattern to match '[Agent Name] is typing'
    r"thank you for your help \w+",  # Pattern to match 'thank you for your help [Agent Name]'



    # ...
]

In [ ]:
import pandas as pd
import re

def clean_column(df, column_name):
    # Define static strings to remove
    strings_to_remove = {
        "Please wait while you are connected to an agent",
        "You are now connected with Medicare.gov Live Chat",
        "Can you please verify your first and last name?",
        "User is Typing",
        "Agent is Typing",
        "Thank you",
        "sir",
        "The chat session is being wrapped up",
        "Thank you for contacting Medicare.gov Live Chat",
        "for contacting Medicare.gov Live Chat",
        "You are now connected with Medicare.gov Live Chat Thank you for contacting Medicare.gov Live Chat",
        "We are here to help you 24 hours a day, 7 days a week",
        "Left the session",
        "The chat session has ended",
        "All times in the above transcript are in the following time zone: (GMT-05:00) Indiana (East)",
        "It appears that you can’t see my reply, so I’m going to end the chat",
        "If you can see this message and you still need assistance",
        "please initiate a new chat or call 1-800-MEDICARE (1-800-633-4227)",
        "Representatives are available to assist you 24 hours a day, 7 days a week",
        "My pleasure",
        # "Thank you",
        "you too",
        # "thank you",
        "For privacy purposes, please do not disclose any personal information such as your Social Security Number, Medicare ID, or any other sensitive medical or personal information",
        "are you still there?",
        "I am still researching that information",
        "It will take me just a moment to review and respond to your question",
        "Thank you for your patience",
        "One moment please while I look that up",
        "One moment",
        "Hello",
        "Hi",
        "TTY users can call 1-877-486-2048",
        "I'll be happy to assist you today"
        # Add more static replacements as needed
    }

    # Check if the column exists in the dataframe
    if column_name not in df.columns:
        raise ValueError(f"The column {column_name} does not exist in the dataframe.")

    # Copy the column to avoid modifying the original data
    cleaned_column = df[column_name].copy()

    # Replace /n and <br> with ' '
    cleaned_column = cleaned_column.str.replace("\n", "", regex=False)
    cleaned_column = cleaned_column.str.replace(r"<br>", ' ', regex=True)

    # Remove static strings
    for string in strings_to_remove:
        cleaned_column = cleaned_column.str.replace(string, '', regex=False)

    # Return the cleaned column
    return cleaned_column

# Usage:
# df['clean_chat'] = clean_column(df, 'Chat')
# df['clean_topic'] = clean_column(df, 'Topic')


In [ ]:
import pandas as pd
import re

def apply_regex_replacements(df, column_name, regex_patterns):
    # Check if the column exists in the dataframe
    if column_name not in df.columns:
        raise ValueError(f"The column {column_name} does not exist in the dataframe.")

    # Copy the column to avoid modifying the original data
    cleaned_column = df[column_name].copy()

    # Apply regex pattern replacements
    for pattern in regex_patterns:
        cleaned_column = cleaned_column.str.replace(pattern, '', regex=True)

    # Return the cleaned column
    return cleaned_column

# Usage:
# regex_patterns = [
#     r'My name is \[.*?\]',  # Pattern to match 'My name is [Agent Name]'
#     # ... (include all other patterns to remove here)
#     r"thank you for your help \w+"  # Pattern to match 'thank you for your help [Agent Name]'
# ]
# df['clean_chat'] = apply_regex_replacements(df, 'clean_chatclean', regex_patterns)


In [ ]:
import re

def clean_text(text):
    # Remove all punctuation and digits
    text = re.sub(r'[^\w\s]', '', text)  # Removes punctuation
    text = re.sub(r'\d', '', text)       # Removes digits
    # Split the text into words, filter out non-alphabetic words, and rejoin the words
    cleaned = ' '.join(w for w in text.split() if w.isalpha())
    return cleaned

# Usage
# df['clean_chat'] = df['clean_chat'].apply(clean_text)
# df['clean_topic'] = df['clean_topic'].apply(clean_text)


In [ ]:
import os
import random
import pandas as pd

def read_text_files_to_df(directory_path, num_files_to_sample=None):
    # Get a list of all text files (with .txt extension) in the directory
    all_files = [f for f in os.listdir(directory_path) if os.path.isfile(os.path.join(directory_path, f)) and f.endswith('.txt')]

    # Select files
    if num_files_to_sample is not None:
        num_files_to_select = min(num_files_to_sample, len(all_files))
        selected_files = random.sample(all_files, num_files_to_select)
    else:
        selected_files = all_files  # Use all files if no specific sample size is provided

    # Read the selected files and store filename, topic, and chat content
    data = []
    for file in selected_files:
        with open(os.path.join(directory_path, file), 'r', encoding='utf-8') as f:
            lines = f.readlines()
            topic = lines[0].strip()  # First line is the topic
            chat = ' '.join(lines[1:]).strip()  # Rest of the lines are the chat
            filename = os.path.splitext(file)[0]  # Filename without extension
            data.append({'filename': filename, 'topic': topic, 'chat': chat})

    # Convert the list of dictionaries to a DataFrame
    df_all = pd.DataFrame(data)
    return df_all

# Usage:
# directory_path = '/opt/app/agent-assist/data/chat/transcripts/2022-10'
# df_all = read_text_files_to_df(directory_path)  # Read all files into a DataFrame
# df_sample = read_text_files_to_df(directory_path, num_files_to_sample=10000)  # Read a sample of 10,000 files into a DataFrame


In [ ]:
# tokenize and fileter for stop words, also convert to lower case before processing.
def tokenize_and_filter(text):
    tokens = word_tokenize(text.lower())
    # return tokens
    return [w for w in tokens if w not in stop_words]

In [ ]:
import pandas as pd
from nltk.util import ngrams

def add_ngram_columns(df):
    # Define a function to generate n-grams from tokens
    def generate_ngrams(tokens, n):
        return [' '.join(ngram) for ngram in ngrams(tokens, n)]

    # Check if 'topic_tokens' column exists
    if 'topic_tokens' not in df.columns:
        raise ValueError("The dataframe does not contain a 'topic_tokens' column.")

    # Apply the generate_ngrams function to each row in the DataFrame for each n value
    df['unigrams'] = df['topic_tokens'].apply(lambda tokens: generate_ngrams(tokens, 1))
    df['bigrams'] = df['topic_tokens'].apply(lambda tokens: generate_ngrams(tokens, 2))
    df['trigrams'] = df['topic_tokens'].apply(lambda tokens: generate_ngrams(tokens, 3))
    df['quadgrams'] = df['topic_tokens'].apply(lambda tokens: generate_ngrams(tokens, 4))

    return df

# Usage:
# df = add_ngram_columns(df)


In [ ]:
import pandas as pd
from nltk import FreqDist

def compute_ngram_freq_dist(df, *columns):
    freq_dists = {}
    for column in columns:
        # Check if column exists
        if column not in df.columns:
            raise ValueError(f"The dataframe does not contain a '{column}' column.")

        # Flatten the list of lists to a single list of n-grams
        ngrams_list = [ngram for sublist in df[column] for ngram in sublist]

        # Compute the frequency distribution of the n-grams
        freq_dist = FreqDist(ngrams_list)
        freq_dists[column] = freq_dist

    return freq_dists

# Usage:
# Assuming df has 'unigrams', 'bigrams', 'trigrams', 'quadgrams' columns
# freq_dists = compute_ngram_freq_dist(df, 'unigrams', 'bigrams', 'trigrams', 'quadgrams')


In [ ]:
# Configure display options
# pd.set_option('display.max_colwidth', None)  # Display full content of each column
# pd.set_option('display.max_rows', None)  # Display all rows

In [ ]:
import pandas as pd

def create_training_data(df, freq_dists, keywords, top_n, max_rows_per_ngram):
    # Initialize list to hold the data
    training_data = []

    # Iterate through each ngram type
    for ngram_type in ['unigrams', 'bigrams', 'trigrams', 'quadgrams']:
        # Get the top_n ngrams and their frequencies
        ngrams_freqs = freq_dists[ngram_type].most_common(top_n)

        # Iterate through each keyword
        for keyword in keywords:
            # Filter ngrams that contain the keyword exclusively
            filtered_ngrams = [ngram for ngram, freq in ngrams_freqs if keyword in ngram and all(other_keyword not in ngram for other_keyword in keywords if other_keyword != keyword)]

            # For each filtered ngram, find the rows in df where this ngram appears
            for ngram in filtered_ngrams:
                ngram_str = ' '.join(ngram)
                matched_rows = df[df[ngram_type].apply(lambda x: ngram_str in [' '.join(y) for y in x])]

                # Limit the matched rows to max_rows_per_ngram
                limited_matched_rows = matched_rows.head(max_rows_per_ngram)

                # For each matched row, store the keyword, ngram, and topic
                for _, row in limited_matched_rows.iterrows():
                    training_data.append({
                        'Keyword': keyword,
                        'Ngram': ngram_str,
                        'Topic': row['topic'],
                        'Filename': row['filename']  # Assuming there's a 'filename' column
                    })

    # Convert the training data into a DataFrame
    training_df = pd.DataFrame(training_data, columns=['Keyword', 'Ngram', 'Topic', 'Filename'])
    training_df['Ngram'] = training_df['Ngram'].str.replace(r'(?<=\b\w) (?=\w\b)', '', regex=True).str.replace(r'\s{2,}', ' ', regex=True)

    return training_df

# Usage:
# df = your dataframe with columns: 'filename', 'topic', 'unigrams', 'bigrams', 'trigrams', 'quadgrams'
# freq_dists = your frequency distributions for 'unigrams', 'bigrams', 'trigrams', 'quadgrams'
# keywords = ['claim', 'another_keyword', ...]
# training_df = create_training_data(df, freq_dists, keywords, top_n=1000, max_rows_per_ngram=10)
# print(training_df)
